Our approach is RAM intensive, so the input data should be less than 1000 images.

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
%cd /content/drive/MyDrive/CSAW-HackML-2020-master_new

/content/drive/MyDrive/CSAW-HackML-2020-master_new


In [15]:
import keras
import sys
import h5py
import numpy as np
from PIL import Image
import numpy as np
from collections import Counter
from statistics import mode,median
from sklearn.metrics import confusion_matrix

In [16]:
def data_loader(filepath):
    data = h5py.File(filepath, 'r')
    x_data = np.array(data['data'])
    y_data = np.array(data['label'])
    x_data = x_data.transpose((0,2,3,1))
    return x_data, y_data

def data_preprocess(x_data):
    return x_data/255 

In [17]:
bd_model = keras.models.load_model('models/sunglasses_bd_net.h5')

In [18]:
x, y_clean = data_loader('data/clean_validation_data.h5')

In [19]:
bs = 100

all_x_cleans = []
for i in range((x.shape[0])//bs):
    all_x_cleans.append(data_preprocess(x[i*bs : i*bs + bs]))



all_x_cleans.append(x[(x.shape[0]//bs)*100:])    

X_clean = np.concatenate(all_x_cleans, axis=0)
X_clean.shape

(11547, 55, 47, 3)

In [20]:
x, y_poison = data_loader('data/sunglasses_poisoned_data.h5')

In [21]:
bs = 100

all_x_poisons = []
for i in range((x.shape[0])//bs):
    all_x_poisons.append(data_preprocess(x[i*bs : i*bs + bs]))

all_x_poisons.append(x[(x.shape[0]//bs)*100:])    

X_poison = np.concatenate(all_x_poisons, axis=0)
X_poison.shape

(12830, 55, 47, 3)

<h2>First part</h2>

In [22]:
img_wt_1 = 0.50
img_wt_2 = 0.50
num_pert = 50
samples = 1000
samples_poisoned = 1000

Function for combining two images to generate perturbed images.

In [23]:
import cv2
def combine(img1,img2):
    img1 = img1.astype('float32')
    img2 = img2.astype('float32')

    return cv2.addWeighted(img1, img_wt_1, img2, img_wt_2, 0)

In [24]:
def random_samples(X):
    return X[np.random.randint(0,len(X))]

In [25]:
data = X_clean[0:samples]

In [26]:
combined_five = []

for x in data:
    rand_idx = []
    for i in range(num_pert):
        rand_idx.append(np.random.randint(0, len(X_clean)))

    for idx in rand_idx:
        combined_five.append(combine(x, X_clean[idx]))

In [27]:
all_outs = []
bs = 100

for i in range(0,int(len(combined_five)/bs)):
    combined_five_np = np.array(combined_five[i*bs:i*bs + bs])
    all_outs.append(bd_model(combined_five_np))

res_max = np.concatenate(all_outs, axis=0 )
res_max_argmax = np.argmax(res_max, axis=1)

k = []
for i in range(samples):
    k.append(Counter(res_max_argmax[i*num_pert:(i+1)*num_pert]))


a = []
for i in k:
    max_res = 0
    for key in i:
        max_res = max(max_res, i[key])
    a.append(max_res)

p = np.percentile(a,96)    

<h2>second part</h2>


In [28]:
# To our program, we created a test dataset of size 2000 with 1000 clean and poisoned data each. Replace data with your own dataset

data = np.concatenate((X_clean[:samples],X_poison[:samples_poisoned]))
labels = np.concatenate((np.ones(samples),np.zeros(samples_poisoned)))  # 1 if clean, 0 if poisoned

# Actual labels with N+1 class
temp = 1283*np.ones(samples_poisoned)
true_labels = np.concatenate((y_clean[:samples],temp))

In [29]:
combined_five = []

for x in data:
    rand_idx = []
    for i in range(num_pert):
        rand_idx.append(np.random.randint(0, len(X_clean)))    

    for idx in rand_idx:
        combined_five.append(combine(x, X_clean[idx]))

In [30]:
all_outs = []
bs = 100

for i in range(0,int(len(combined_five)/bs)):
    combined_five_np = np.array(combined_five[i*bs:i*bs + bs])
    all_outs.append(bd_model(combined_five_np))

res_max = np.concatenate(all_outs, axis=0 )
res_max_argmax = np.argmax(res_max, axis=1)

k = []
for i in range(len(data)):
    k.append(Counter(res_max_argmax[i*num_pert:(i+1)*num_pert]))


a = []
for i in k:
    max_res = 0
    for key in i:
        max_res = max(max_res, i[key])
    a.append(max_res)

In [31]:
y_pred = []
for i in range((len(data))):
    if a[i] > p:
        y_pred.append(0)
    else:
        y_pred.append(1)


temp = 0
for i in range(len(data)):
    if labels[i] == y_pred[i]:
            temp += 1

print(temp/(len(data)))

# Accuracy for classifying inputs into poisoned and clean classes
conf = confusion_matrix(labels,y_pred)
print(conf)

0.9545
[[946  54]
 [ 37 963]]


In [32]:
y_pred_actual = np.argmax(bd_model(data),axis=1)

for i in range(len(data)):
    if y_pred[i] == 0:
        y_pred_actual[i] = 1283 

temp = 0
for i in range(len(data)):
    if true_labels[i] == y_pred_actual[i]:
            temp += 1

# Accuracy for classifying inputs into all seperate clean classes(classes 0 to 1282) and poisoned inputs(class 1283)
print(temp/(len(data)))  

0.943


FAR - False acceptance rate

FRR - False rejection rate

As we vary percentile from 95 to 99, FRR increases and FAR decreases. 

In [33]:
FRR = conf[0][1]/(conf[0][0] + conf[0][1])
FAR = conf[1][0]/(conf[1][1] + conf[1][0])
print(f'FRR = {FRR*100}%, FAR = {FAR*100}%')

FRR = 5.4%, FAR = 3.6999999999999997%


In [34]:
p

25.0